![logo_universidad](https://bloqueneon.uniandes.edu.co//content/enforced/171175-202310_ISIS3301_02/morado/assets/img/banner.png?_&d2lSessionVal=d1HNZaZihV1jOwJNVtWDIWG7y&ou=171175)

## Laboratorio 1 - Clustering

El objetivo de este notebook es realizar el analisis del caso BiciAlpes, para el cual se tiene un conjunto de datos que contiene información de los accidentes ocurridos involucrando bicicletas en la ciudad de Bogotá. El objetivo es realizar un análisis exploratorio de los datos para identificar patrones y relaciones entre las variables, y así poder responder las preguntas de negocio planteadas. Adicionalmente, se espera usar 3 algoritmos de agrupamiento para identificar grupos de accidentes que se puedan utilizar para responder las preguntas de negocio.

Por medio de 3 iteraciones, se espera que el grupo desarrolle un modelo de agrupamiento que permita responder las preguntas de negocio. En cada iteración se espera que nuestro equipo de trabajo realice los siguientes pasos:

* **Iteración1.** Analisis exploratorio de los datos y creación de un primer modelo de agrupamiento.
* **Iteración2.** Segundo modelo con mejoras a nivel de preparación de datos y configuración de hiper-parámetros
* **Iteración3.** Modelo final que incluye mejoras identificadas al revisar mejor la literatura y la documentación de los algoritmos de agrupamiento.

### Descripción de negocio

BiciAlpes es una organización que presta un servicio de alquiler de bicicletas, solidario y con muchos beneficios para los usuarios. Su objetivo es incentivar planes de movilidad sostenible en zonas urbanas, que contribuyan a reducir la emisión de gases de efecto invernadero que afectan la temperatura del planeta. En el último año, han detectado con preocupación que el uso de la bicicleta se está viendo obstaculizado por la falta de información sobre la seguridad en las vías. Piensa que proporcionar a los ciclistas información sobre las rutas más seguras podría contribuir a aumentar la práctica de esta actividad. Así, decide emprender un estudio para conocer cuáles son los factores que más impactan en los accidentes viales que involucran ciclistas. Este conocimiento, además, podría ayudar a las autoridades y planificadores urbanos en la implementación de mecanismos que reduzcan la ocurrencia de accidentes, así como otros planes de movilidad sostenible. Para realizar este estudio BiciAlpes han recopilado datos, de fuentes abiertas de la Alcaldía, sobre accidentes en los cuales se han visto involucrados ciclistas.

BiciAlpes los ha contactado para realizar este proceso de caracterización e identificación de patrones que puedan ser utilizados para entender los factores que indicen en la severidad de los accidentes, con el fin de trazar mapas de seguridad vial para los ciclistas.  Los datos, al igual que el diccionario de datos, los encuentra disponibles en este directorio: [datos](./datos/Datos_BiciAlpes.csv)

## Referencias

1. [Datos a utilizar: Datos_BiciAlpes.csv](./datos/Datos_BiciAlpes.csv)
2. [Diccionario de datos: Diccionario_BiciAlpes.csv](./datos/Diccionario_BiciAlpes.xlsx)
3. [Documentación de los algoritmos de agrupamiento](https://scikit-learn.org/stable/modules/clustering.html)
4. [Documentación de la librería de visualización de datos](https://seaborn.pydata.org/)
5. [Informacion del negocio](./Laboratorio1_enunciado.md)

## 1. Carga de librerias necesarias

In [1]:
#Se cargan las librerias necesarias para la implementacion
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()  # for plot styling

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.mixture._gaussian_mixture import GaussianMixture

from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D # for 3D plots

#Scipy para aglomeracion
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.cluster.hierarchy import fcluster

## 2. Carga de datos

In [5]:
# Se cargan los datos necesarios para la implementacion
df_accidents = pd.read_csv('./datos/Datos_BiciAlpes.csv', sep=';', encoding = 'latin-1', index_col=0)

In [8]:
# Se muestran una fraccion de los datos
df_accidents.head()

,Number_of_Casualties,Day_of_Week,Road_Type,Speed_limit,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Urban_or_Rural_Area,Vehicle_Type,Did_Police_Officer_Attend_Scene_of_Accident,Junction_Detail,Number_of_Vehicles,Accident_Severity,Unnamed: 14
Time,,,,,,,,,,,,,,
Mañana,1,Día laboral,6,30.0,1,1,1,1,bike,1,3,1,3,NaN
Mañana,1,Día laboral,6,30.0,1,1,1,1,bike,1,0,1,3,NaN
Tarde,1,Fin de semana,6,30.0,1,1,1,1,bike,1,3,1,3,NaN
Tarde,2,Día laboral,6,30.0,1,1,1,1,bike,1,6,1,2,NaN
Mañana,2,Día laboral,6,30.0,1,1,1,1,bike,1,6,1,3,NaN
